# 2层神经网络从零实现

这个notebook将实现一个完整的2层神经网络，包括：
- 前向传播: `Linear -> ReLU -> Linear -> Softmax`
- 反向传播: 手动实现交叉熵损失的反向传播
- 梯度检查: 数值验证梯度计算的正确性

## 📚 学习目标
- 理解神经网络的前向传播和反向传播
- 掌握手动实现梯度计算
- 学会梯度检查技术
- 实现完整的训练循环


In [ ]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# 设置随机种子
torch.manual_seed(42)
np.random.seed(42)

print("导入完成！")


## 1. 数据准备


## 🔍 Xavier初始化详解 (Xavier Initialization Explanation)

### 什么是Xavier初始化？

```python
# 这两行代码的作用：
torch.nn.init.xavier_uniform_(self.W1)  # 初始化第一层权重
torch.nn.init.xavier_uniform_(self.W2)  # 初始化第二层权重
```

### 为什么需要Xavier初始化？

**问题：**
- **梯度消失 (Vanishing Gradients)**: 梯度在反向传播过程中逐渐变小，导致深层网络难以训练
- **梯度爆炸 (Exploding Gradients)**: 梯度在反向传播过程中逐渐变大，导致训练不稳定

**解决方案：**
Xavier初始化通过控制权重分布的方差来保持信号在前向和反向传播过程中的强度。

### Xavier初始化的数学原理

**目标：** 保持每层输出的方差与输入的方差相近

**公式：**
- 对于均匀分布：`W ~ U(-a, a)`，其中 `a = sqrt(6/(fan_in + fan_out))`
- 对于正态分布：`W ~ N(0, 2/(fan_in + fan_out))`

**其中：**
- `fan_in`: 输入神经元数量
- `fan_out`: 输出神经元数量

### 实际效果

**不使用Xavier初始化：**
- 权重可能过大或过小
- 激活值可能饱和
- 梯度传播不稳定

**使用Xavier初始化：**
- 权重分布合理
- 激活值在有效范围内
- 梯度传播稳定


In [ ]:
# Xavier初始化效果可视化 (Xavier Initialization Visualization)

import matplotlib.pyplot as plt
import numpy as np

# 设置参数
fan_in = 2      # 输入神经元数量
fan_out = 64    # 输出神经元数量
num_samples = 10000

# 1. 随机初始化（不使用Xavier）
random_weights = torch.randn(fan_in, fan_out)

# 2. Xavier初始化
xavier_weights = torch.randn(fan_in, fan_out)
torch.nn.init.xavier_uniform_(xavier_weights)

# 3. 可视化权重分布
plt.figure(figsize=(15, 5))

# 随机初始化的权重分布
plt.subplot(1, 3, 1)
plt.hist(random_weights.flatten().detach().numpy(), bins=50, alpha=0.7, color='red')
plt.title('Random Initialization')
plt.xlabel('Weight Value')
plt.ylabel('Frequency')
plt.grid(True, alpha=0.3)

# Xavier初始化的权重分布
plt.subplot(1, 3, 2)
plt.hist(xavier_weights.flatten().detach().numpy(), bins=50, alpha=0.7, color='blue')
plt.title('Xavier Initialization')
plt.xlabel('Weight Value')
plt.ylabel('Frequency')
plt.grid(True, alpha=0.3)

# 对比图
plt.subplot(1, 3, 3)
plt.hist(random_weights.flatten().detach().numpy(), bins=50, alpha=0.5, color='red', label='Random')
plt.hist(xavier_weights.flatten().detach().numpy(), bins=50, alpha=0.5, color='blue', label='Xavier')
plt.title('Comparison')
plt.xlabel('Weight Value')
plt.ylabel('Frequency')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# 4. 统计信息对比
print("📊 权重统计信息对比:")
print("-" * 50)
print(f"随机初始化:")
print(f"  均值: {random_weights.mean().item():.4f}")
print(f"  标准差: {random_weights.std().item():.4f}")
print(f"  最小值: {random_weights.min().item():.4f}")
print(f"  最大值: {random_weights.max().item():.4f}")

print(f"\nXavier初始化:")
print(f"  均值: {xavier_weights.mean().item():.4f}")
print(f"  标准差: {xavier_weights.std().item():.4f}")
print(f"  最小值: {xavier_weights.min().item():.4f}")
print(f"  最大值: {xavier_weights.max().item():.4f}")

# 5. 理论Xavier范围
xavier_range = np.sqrt(6.0 / (fan_in + fan_out))
print(f"\n🎯 理论Xavier范围: ±{xavier_range:.4f}")
print(f"实际Xavier范围: ±{xavier_weights.abs().max().item():.4f}")


In [ ]:
# 生成分类数据
X, y = make_classification(
    n_samples=1000,
    n_features=4,
    n_redundant=0,
    n_informative=4,
    n_classes=3,
    random_state=42
)

print(f"数据形状: X={X.shape}, y={y.shape}")
print(f"类别分布: {np.bincount(y)}")

# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# 标准化数据
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 转换为PyTorch张量
X_train_tensor = torch.FloatTensor(X_train_scaled)
X_test_tensor = torch.FloatTensor(X_test_scaled)
y_train_tensor = torch.LongTensor(y_train)
y_test_tensor = torch.LongTensor(y_test)

print(f"训练集: {X_train_tensor.shape}, 测试集: {X_test_tensor.shape}")

# 可视化数据 (使用前两个特征)
plt.figure(figsize=(8, 6))
scatter = plt.scatter(X_train_scaled[:, 0], X_train_scaled[:, 1], 
                     c=y_train, cmap='viridis', alpha=0.7)
plt.colorbar(scatter)
plt.xlabel('特征 1')
plt.ylabel('特征 2')
plt.title('训练数据分布')
plt.show()


## 2. 2层神经网络实现


In [ ]:
class TwoLayerNN:
    """
    2层神经网络实现
    结构: Linear -> ReLU -> Linear -> Softmax
    """
    
    def __init__(self, input_size, hidden_size, output_size, learning_rate=0.01):
        # 网络参数
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.learning_rate = learning_rate
        
        # 初始化权重和偏置
        self.W1 = torch.randn(input_size, hidden_size, requires_grad=True)
        self.b1 = torch.zeros(hidden_size, requires_grad=True)
        self.W2 = torch.randn(hidden_size, output_size, requires_grad=True)
        self.b2 = torch.zeros(output_size, requires_grad=True)
        
        # Xavier初始化
        torch.nn.init.xavier_uniform_(self.W1)
        torch.nn.init.xavier_uniform_(self.W2)
        
        print(f"网络结构: {input_size} -> {hidden_size} -> {output_size}")
        print(f"学习率: {learning_rate}")
    
    def forward(self, X):
        """前向传播"""
        # 第一层: Linear -> ReLU
        self.z1 = X @ self.W1 + self.b1
        self.a1 = torch.relu(self.z1)
        
        # 第二层: Linear -> Softmax
        self.z2 = self.a1 @ self.W2 + self.b2
        
        # Softmax with numerical stability
        exp_scores = torch.exp(self.z2 - torch.max(self.z2, dim=1, keepdim=True)[0])
        self.probs = exp_scores / torch.sum(exp_scores, dim=1, keepdim=True)
        
        return self.probs
    
    def backward(self, X, y):
        """反向传播"""
        batch_size = X.size(0)
        
        # 计算交叉熵损失
        # 将标签转换为one-hot编码
        y_one_hot = torch.zeros_like(self.probs)
        y_one_hot.scatter_(1, y.unsqueeze(1), 1)
        
        # 损失函数: -log(prob_correct_class)
        loss = -torch.mean(torch.sum(y_one_hot * torch.log(self.probs + 1e-8), dim=1))
        
        # 反向传播计算梯度
        # 输出层梯度
        dz2 = (self.probs - y_one_hot) / batch_size
        
        # 第二层参数梯度
        dW2 = self.a1.T @ dz2
        db2 = torch.sum(dz2, dim=0)
        
        # 第一层激活梯度
        da1 = dz2 @ self.W2.T
        
        # ReLU梯度
        dz1 = da1 * (self.z1 > 0).float()
        
        # 第一层参数梯度
        dW1 = X.T @ dz1
        db1 = torch.sum(dz1, dim=0)
        
        # 更新参数
        with torch.no_grad():
            self.W1 -= self.learning_rate * dW1
            self.b1 -= self.learning_rate * db1
            self.W2 -= self.learning_rate * dW2
            self.b2 -= self.learning_rate * db2
        
        return loss.item()
    
    def predict(self, X):
        """预测"""
        with torch.no_grad():
            probs = self.forward(X)
            predictions = torch.argmax(probs, dim=1)
        return predictions
    
    def accuracy(self, X, y):
        """计算准确率"""
        predictions = self.predict(X)
        return torch.mean((predictions == y).float()).item()

# 创建网络实例
model = TwoLayerNN(input_size=4, hidden_size=10, output_size=3, learning_rate=0.01)


In [ ]:
# 训练参数
epochs = 1000
print_every = 100

# 训练循环
train_losses = []
train_accuracies = []
test_accuracies = []

print("开始训练...")
for epoch in range(epochs):
    # 前向传播
    model.forward(X_train_tensor)
    
    # 反向传播
    loss = model.backward(X_train_tensor, y_train_tensor)
    
    # 计算准确率
    train_acc = model.accuracy(X_train_tensor, y_train_tensor)
    test_acc = model.accuracy(X_test_tensor, y_test_tensor)
    
    # 记录指标
    train_losses.append(loss)
    train_accuracies.append(train_acc)
    test_accuracies.append(test_acc)
    
    # 打印进度
    if epoch % print_every == 0:
        print(f"Epoch {epoch:4d} | Loss: {loss:.4f} | Train Acc: {train_acc:.4f} | Test Acc: {test_acc:.4f}")

print(f"\n训练完成！最终测试准确率: {test_accuracies[-1]:.4f}")


In [ ]:
# 绘制训练曲线
plt.figure(figsize=(15, 5))

# 损失曲线
plt.subplot(1, 3, 1)
plt.plot(train_losses)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('训练损失')
plt.grid(True)

# 准确率曲线
plt.subplot(1, 3, 2)
plt.plot(train_accuracies, label='训练准确率')
plt.plot(test_accuracies, label='测试准确率')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('准确率变化')
plt.legend()
plt.grid(True)

# 混淆矩阵
plt.subplot(1, 3, 3)
predictions = model.predict(X_test_tensor)
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test_tensor.numpy(), predictions.numpy())

plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
plt.title('混淆矩阵')
plt.colorbar()
tick_marks = np.arange(len(np.unique(y_test)))
plt.xticks(tick_marks, np.unique(y_test))
plt.yticks(tick_marks, np.unique(y_test))

# 添加数值标注
thresh = cm.max() / 2.
for i, j in np.ndindex(cm.shape):
    plt.text(j, i, format(cm[i, j], 'd'),
             ha="center", va="center",
             color="white" if cm[i, j] > thresh else "black")

plt.tight_layout()
plt.show()

print(f"最终训练准确率: {train_accuracies[-1]:.4f}")
print(f"最终测试准确率: {test_accuracies[-1]:.4f}")


## 5. 梯度检查 (Gradient Checking)


In [ ]:
def gradient_check(model, X, y, epsilon=1e-7):
    """
    梯度检查函数
    通过数值微分验证解析梯度的正确性
    """
    # 创建模型副本用于梯度检查
    model_check = TwoLayerNN(model.input_size, model.hidden_size, model.output_size, model.learning_rate)
    
    # 复制参数
    model_check.W1.data = model.W1.data.clone()
    model_check.b1.data = model.b1.data.clone()
    model_check.W2.data = model.W2.data.clone()
    model_check.b2.data = model.b2.data.clone()
    
    # 前向传播计算解析梯度
    model_check.forward(X)
    loss_original = model_check.backward(X, y)
    
    # 获取解析梯度
    grad_W1_analytic = model_check.W1.grad.clone()
    grad_b1_analytic = model_check.b1.grad.clone()
    grad_W2_analytic = model_check.W2.grad.clone()
    grad_b2_analytic = model_check.b2.grad.clone()
    
    print("开始梯度检查...")
    
    # 检查W1的梯度
    grad_W1_numerical = torch.zeros_like(model_check.W1)
    for i in range(model_check.W1.size(0)):
        for j in range(model_check.W1.size(1)):
            # 创建模型副本
            model_plus = TwoLayerNN(model.input_size, model.hidden_size, model.output_size, model.learning_rate)
            model_plus.W1.data = model_check.W1.data.clone()
            model_plus.b1.data = model_check.b1.data.clone()
            model_plus.W2.data = model_check.W2.data.clone()
            model_plus.b2.data = model_check.b2.data.clone()
            
            # 增加epsilon
            model_plus.W1.data[i, j] += epsilon
            loss_plus = model_plus.backward(X, y)
            
            # 减少epsilon
            model_minus = TwoLayerNN(model.input_size, model.hidden_size, model.output_size, model.learning_rate)
            model_minus.W1.data = model_check.W1.data.clone()
            model_minus.b1.data = model_check.b1.data.clone()
            model_minus.W2.data = model_check.W2.data.clone()
            model_minus.b2.data = model_check.b2.data.clone()
            
            model_minus.W1.data[i, j] -= epsilon
            loss_minus = model_minus.backward(X, y)
            
            # 计算数值梯度
            grad_W1_numerical[i, j] = (loss_plus - loss_minus) / (2 * epsilon)
    
    # 计算相对误差
    diff_W1 = torch.abs(grad_W1_analytic - grad_W1_numerical)
    rel_error_W1 = diff_W1 / (torch.abs(grad_W1_analytic) + torch.abs(grad_W1_numerical) + epsilon)
    max_error_W1 = torch.max(rel_error_W1).item()
    
    print(f"W1梯度检查:")
    print(f"  解析梯度形状: {grad_W1_analytic.shape}")
    print(f"  数值梯度形状: {grad_W1_numerical.shape}")
    print(f"  最大相对误差: {max_error_W1:.2e}")
    print(f"  梯度检查通过: {'✅' if max_error_W1 < 1e-5 else '❌'}")
    
    return max_error_W1 < 1e-5

# 使用小批量数据进行梯度检查
X_small = X_train_tensor[:10]  # 使用前10个样本
y_small = y_train_tensor[:10]

gradient_check_passed = gradient_check(model, X_small, y_small)
print(f"\n梯度检查结果: {'通过' if gradient_check_passed else '失败'}")


## 6. 与PyTorch内置模块对比


In [ ]:
# 使用PyTorch内置模块构建相同结构的网络
class PyTorchNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(PyTorchNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)
        self.relu = nn.ReLU()
        
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# 创建PyTorch模型
pytorch_model = PyTorchNN(4, 10, 3)

# 复制我们手动实现的权重
with torch.no_grad():
    pytorch_model.fc1.weight.copy_(model.W1.T)  # 注意转置
    pytorch_model.fc1.bias.copy_(model.b1)
    pytorch_model.fc2.weight.copy_(model.W2.T)
    pytorch_model.fc2.bias.copy_(model.b2)

# 测试两个模型是否产生相同结果
test_input = X_test_tensor[:5]
manual_output = model.forward(test_input)
pytorch_output = torch.softmax(pytorch_model(test_input), dim=1)

print("手动实现 vs PyTorch内置模块对比:")
print(f"手动实现输出形状: {manual_output.shape}")
print(f"PyTorch输出形状: {pytorch_output.shape}")
print(f"输出差异: {torch.max(torch.abs(manual_output - pytorch_output)).item():.2e}")

# 使用PyTorch训练
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(pytorch_model.parameters(), lr=0.01)

pytorch_train_losses = []
pytorch_train_accuracies = []

print("\n使用PyTorch训练相同模型...")
for epoch in range(1000):
    optimizer.zero_grad()
    outputs = pytorch_model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()
    
    pytorch_train_losses.append(loss.item())
    
    # 计算准确率
    with torch.no_grad():
        _, predicted = torch.max(outputs.data, 1)
        accuracy = (predicted == y_train_tensor).float().mean()
        pytorch_train_accuracies.append(accuracy.item())
    
    if epoch % 200 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item():.4f}, Acc: {accuracy.item():.4f}")

# 比较最终性能
manual_final_acc = model.accuracy(X_test_tensor, y_test_tensor)
with torch.no_grad():
    pytorch_test_output = pytorch_model(X_test_tensor)
    _, pytorch_predicted = torch.max(pytorch_test_output, 1)
    pytorch_final_acc = (pytorch_predicted == y_test_tensor).float().mean().item()

print(f"\n最终测试准确率对比:")
print(f"手动实现: {manual_final_acc:.4f}")
print(f"PyTorch内置: {pytorch_final_acc:.4f}")
print(f"差异: {abs(manual_final_acc - pytorch_final_acc):.4f}")


## 7. 总结

通过这个notebook，我们成功实现了：

### ✅ 完成的功能
1. **完整的2层神经网络**: Linear -> ReLU -> Linear -> Softmax
2. **手动实现前向传播**: 包括数值稳定的Softmax
3. **手动实现反向传播**: 交叉熵损失的反向传播
4. **梯度检查**: 数值验证梯度计算的正确性
5. **完整训练循环**: 包括损失和准确率监控
6. **可视化结果**: 训练曲线、混淆矩阵
7. **与PyTorch对比**: 验证实现正确性

### 🎯 关键学习点
- **数值稳定性**: Softmax计算中减去最大值避免溢出
- **梯度计算**: 理解每一层的梯度传播过程
- **梯度检查**: 验证手动实现的正确性
- **Xavier初始化**: 帮助网络更好地训练

### 🚀 扩展练习
- 尝试不同的激活函数 (Tanh, LeakyReLU)
- 添加Dropout层防止过拟合
- 实现更多的层数
- 添加批归一化层
- 使用不同的优化器 (Adam, RMSprop)

这个实现为理解深度学习的基础原理提供了坚实的基础！


# 📝 神经网络编程挑战 (Neural Network Coding Challenges)

以下每个cell都是一个独立的编程挑战，请根据要求完成代码实现。


## 挑战1: 3层神经网络实现 (3-Layer Neural Network)

**要求 (Requirements):**
1. 修改网络为3层：Linear -> ReLU -> Linear -> ReLU -> Linear -> Softmax
2. 实现完整的前向传播
3. 实现完整的反向传播
4. 比较与2层网络的性能差异

**提示 (Hints):**
- 需要添加第三层权重W3和偏置b3
- 反向传播需要计算额外的梯度
- 注意梯度传播的顺序：从输出层到输入层
- 使用Xavier初始化所有层


In [ ]:
# 在这里实现你的代码
# Write your code here

class ThreeLayerNN:
    """
    3层神经网络实现
    结构: Linear -> ReLU -> Linear -> ReLU -> Linear -> Softmax
    """
    
    def __init__(self, input_size, hidden1_size, hidden2_size, output_size, learning_rate=0.01):
        # 初始化参数
        pass
    
    def forward(self, X):
        """前向传播"""
        pass
    
    def backward(self, X, y):
        """反向传播"""
        pass
    
    def predict(self, X):
        """预测"""
        pass
    
    def accuracy(self, X, y):
        """计算准确率"""
        pass

# 创建3层网络并训练
# Create 3-layer network and train


## 挑战2: Dropout层实现 (Dropout Layer Implementation)

**要求 (Requirements):**
1. 手动实现Dropout层的前向传播和反向传播
2. 在训练时应用Dropout，测试时关闭
3. 分析Dropout对过拟合的影响
4. 比较不同Dropout率的效果

**提示 (Hints):**
- Dropout随机将部分神经元输出设为0
- 训练时：输出 = input * mask / (1-p)，其中mask是伯努利分布
- 测试时：输出 = input（不应用Dropout）
- 反向传播时梯度也要乘以相同的mask


In [ ]:
# 在这里实现你的代码
# Write your code here

class Dropout:
    """
    Dropout层实现
    """
    
    def __init__(self, p=0.5):
        # 初始化Dropout参数
        pass
    
    def forward(self, X, training=True):
        """前向传播"""
        pass
    
    def backward(self, grad_output):
        """反向传播"""
        pass

class TwoLayerNNWithDropout:
    """
    带Dropout的2层神经网络
    """
    
    def __init__(self, input_size, hidden_size, output_size, dropout_p=0.5, learning_rate=0.01):
        # 初始化网络参数和Dropout层
        pass
    
    def forward(self, X, training=True):
        """前向传播"""
        pass
    
    def backward(self, X, y):
        """反向传播"""
        pass
    
    def predict(self, X):
        """预测"""
        pass

# 比较有无Dropout的性能
# Compare performance with and without Dropout


## 挑战3: 批归一化实现 (Batch Normalization Implementation)

**要求 (Requirements):**
1. 手动实现Batch Normalization层
2. 在神经网络中集成批归一化
3. 实现训练和推理模式的区别
4. 分析批归一化对训练稳定性的影响

**提示 (Hints):**
- 批归一化公式：BN(x) = γ * (x - μ) / σ + β
- 训练时使用当前批次的均值和方差
- 推理时使用移动平均的均值和方差
- γ和β是可学习参数


In [ ]:
# 在这里实现你的代码
# Write your code here

class BatchNorm:
    """
    批归一化层实现
    """
    
    def __init__(self, num_features, eps=1e-5, momentum=0.1):
        # 初始化参数
        pass
    
    def forward(self, X, training=True):
        """前向传播"""
        pass
    
    def backward(self, grad_output):
        """反向传播"""
        pass

class TwoLayerNNWithBN:
    """
    带批归一化的2层神经网络
    """
    
    def __init__(self, input_size, hidden_size, output_size, learning_rate=0.01):
        # 初始化网络参数和批归一化层
        pass
    
    def forward(self, X, training=True):
        """前向传播"""
        pass
    
    def backward(self, X, y):
        """反向传播"""
        pass

# 比较有无批归一化的训练效果
# Compare training effects with and without Batch Normalization


## 挑战4: Adam优化器实现 (Adam Optimizer Implementation)

**要求 (Requirements):**
1. 手动实现Adam优化器
2. 替换SGD优化器
3. 比较Adam和SGD的收敛速度
4. 分析Adam的超参数影响

**提示 (Hints):**
- Adam结合了动量和自适应学习率
- 需要维护一阶和二阶矩估计
- 更新公式：θ = θ - α * m̂ / (√v̂ + ε)
- 典型超参数：β1=0.9, β2=0.999, ε=1e-8


In [ ]:
# 在这里实现你的代码
# Write your code here

class AdamOptimizer:
    """
    Adam优化器实现
    """
    
    def __init__(self, params, lr=0.001, beta1=0.9, beta2=0.999, eps=1e-8):
        # 初始化优化器参数
        pass
    
    def step(self):
        """执行一步优化"""
        pass
    
    def zero_grad(self):
        """清零梯度"""
        pass

class TwoLayerNNWithAdam:
    """
    使用Adam优化器的2层神经网络
    """
    
    def __init__(self, input_size, hidden_size, output_size, learning_rate=0.001):
        # 初始化网络参数和Adam优化器
        pass
    
    def forward(self, X):
        """前向传播"""
        pass
    
    def backward(self, X, y):
        """反向传播"""
        pass
    
    def train_step(self, X, y):
        """训练一步"""
        pass

# 比较SGD和Adam的收敛速度
# Compare convergence speed between SGD and Adam


## 挑战5: 完整梯度检查 (Complete Gradient Checking)

**要求 (Requirements):**
1. 实现完整的梯度检查函数，检查所有参数
2. 检查W1, b1, W2, b2的梯度
3. 分析数值梯度和解析梯度的差异
4. 可视化梯度检查结果

**提示 (Hints):**
- 对每个参数分别进行梯度检查
- 使用中心差分公式提高精度
- 计算相对误差：|grad_analytic - grad_numerical| / (|grad_analytic| + |grad_numerical| + eps)
- 误差阈值通常设为1e-5


In [ ]:
# 在这里实现你的代码
# Write your code here

def complete_gradient_check(model, X, y, epsilon=1e-7):
    """
    完整的梯度检查函数
    检查所有参数：W1, b1, W2, b2
    """
    
    # 1. 计算解析梯度
    # Compute analytical gradients
    pass
    
    # 2. 计算数值梯度
    # Compute numerical gradients
    pass
    
    # 3. 比较梯度
    # Compare gradients
    pass
    
    # 4. 返回检查结果
    # Return checking results
    pass

def visualize_gradient_check(results):
    """
    可视化梯度检查结果
    """
    pass

# 执行完整的梯度检查
# Perform complete gradient checking


## 挑战6: PyTorch原生实现对比 (PyTorch Native Implementation Comparison)

**要求 (Requirements):**
1. 使用PyTorch的nn.Module实现相同的2层神经网络
2. 使用nn.Linear, nn.ReLU, nn.CrossEntropyLoss等原生组件
3. 使用torch.optim优化器进行训练
4. 比较自定义实现和PyTorch原生实现的性能

**提示 (Hints):**
- 继承nn.Module类
- 使用nn.Sequential或手动定义层
- 使用torch.optim.SGD或torch.optim.Adam
- 训练循环更简洁，不需要手动计算梯度


In [ ]:
# 在这里实现你的代码
# Write your code here

import torch.nn as nn
import torch.optim as optim

class PyTorchTwoLayerNN(nn.Module):
    """
    使用PyTorch原生组件实现的2层神经网络
    结构: Linear -> ReLU -> Linear -> Softmax
    """
    
    def __init__(self, input_size, hidden_size, output_size):
        super(PyTorchTwoLayerNN, self).__init__()
        # 定义网络层
        pass
    
    def forward(self, x):
        """前向传播"""
        pass

def train_pytorch_model(model, X_train, y_train, X_val, y_val, epochs=1000, lr=0.01):
    """
    使用PyTorch原生组件训练模型
    """
    
    # 1. 定义损失函数和优化器
    # Define loss function and optimizer
    pass
    
    # 2. 训练循环
    # Training loop
    pass
    
    # 3. 返回训练历史
    # Return training history
    pass

def compare_implementations():
    """
    比较自定义实现和PyTorch原生实现的性能
    """
    
    # 1. 创建两个模型
    # Create two models
    pass
    
    # 2. 训练两个模型
    # Train both models
    pass
    
    # 3. 比较性能
    # Compare performance
    pass
    
    # 4. 可视化对比结果
    # Visualize comparison results
    pass

# 执行完整的PyTorch原生实现和对比
# Execute complete PyTorch native implementation and comparison


## 📚 PyTorch原生实现参考 (PyTorch Native Implementation Reference)

以下是一个完整的PyTorch原生实现示例，你可以参考这个来实现上面的挑战：


In [ ]:
# PyTorch原生实现参考代码 (Reference Implementation)
# 这个代码展示了如何使用PyTorch原生组件实现相同的神经网络

import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

class PyTorchTwoLayerNN(nn.Module):
    """
    使用PyTorch原生组件实现的2层神经网络
    结构: Linear -> ReLU -> Linear -> Softmax
    """
    
    def __init__(self, input_size, hidden_size, output_size):
        super(PyTorchTwoLayerNN, self).__init__()
        # 定义网络层
        self.network = nn.Sequential(
            nn.Linear(input_size, hidden_size),  # 第一层：线性变换
            nn.ReLU(),                           # 激活函数
            nn.Linear(hidden_size, output_size)  # 第二层：线性变换
        )
        
        # Xavier初始化
        for layer in self.network:
            if isinstance(layer, nn.Linear):
                nn.init.xavier_uniform_(layer.weight)
                nn.init.zeros_(layer.bias)
    
    def forward(self, x):
        """前向传播"""
        return self.network(x)

def train_pytorch_model(model, X_train, y_train, X_val, y_val, epochs=1000, lr=0.01):
    """
    使用PyTorch原生组件训练模型
    """
    
    # 定义损失函数和优化器
    criterion = nn.CrossEntropyLoss()  # 交叉熵损失（包含Softmax）
    optimizer = optim.SGD(model.parameters(), lr=lr)
    
    # 训练历史记录
    train_losses = []
    train_accuracies = []
    val_accuracies = []
    
    model.train()  # 设置为训练模式
    
    for epoch in range(epochs):
        # 前向传播
        outputs = model(X_train)
        loss = criterion(outputs, y_train)
        
        # 反向传播和优化
        optimizer.zero_grad()  # 清零梯度
        loss.backward()        # 计算梯度
        optimizer.step()       # 更新参数
        
        # 记录训练损失
        train_losses.append(loss.item())
        
        # 计算训练准确率
        with torch.no_grad():
            train_pred = torch.argmax(outputs, dim=1)
            train_acc = torch.mean((train_pred == y_train).float()).item()
            train_accuracies.append(train_acc)
            
            # 计算验证准确率
            model.eval()  # 设置为评估模式
            val_outputs = model(X_val)
            val_pred = torch.argmax(val_outputs, dim=1)
            val_acc = torch.mean((val_pred == y_val).float()).item()
            val_accuracies.append(val_acc)
            model.train()  # 重新设置为训练模式
        
        # 每100个epoch打印一次
        if (epoch + 1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}, '
                  f'Train Acc: {train_acc:.4f}, Val Acc: {val_acc:.4f}')
    
    return train_losses, train_accuracies, val_accuracies

# 使用示例
print("🚀 创建PyTorch原生模型...")
pytorch_model = PyTorchTwoLayerNN(input_size=2, hidden_size=64, output_size=3)

print("📊 开始训练PyTorch原生模型...")
pytorch_losses, pytorch_train_acc, pytorch_val_acc = train_pytorch_model(
    pytorch_model, X_train, y_train, X_val, y_val, epochs=1000, lr=0.01
)

print("✅ PyTorch原生模型训练完成！")


In [ ]:
# 可视化PyTorch原生模型的训练结果
plt.figure(figsize=(15, 5))

# 训练损失
plt.subplot(1, 3, 1)
plt.plot(pytorch_losses, 'b-', label='PyTorch Native')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss Comparison')
plt.legend()
plt.grid(True, alpha=0.3)

# 训练准确率
plt.subplot(1, 3, 2)
plt.plot(pytorch_train_acc, 'b-', label='PyTorch Native (Train)')
plt.plot(pytorch_val_acc, 'b--', label='PyTorch Native (Val)')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Accuracy Comparison')
plt.legend()
plt.grid(True, alpha=0.3)

# 决策边界
plt.subplot(1, 3, 3)
plot_decision_boundary(pytorch_model, X_test, y_test, title='PyTorch Native Decision Boundary')
plt.tight_layout()
plt.show()

# 计算最终性能
pytorch_model.eval()
with torch.no_grad():
    final_outputs = pytorch_model(X_test)
    final_pred = torch.argmax(final_outputs, dim=1)
    final_acc = torch.mean((final_pred == y_test).float()).item()

print(f"🎯 PyTorch原生模型最终测试准确率: {final_acc:.4f}")

# 显示模型结构
print("\n📋 PyTorch原生模型结构:")
print(pytorch_model)


## 🔍 两种实现方式对比分析 (Implementation Comparison Analysis)

### 自定义实现 vs PyTorch原生实现

**自定义实现的优势：**
- ✅ 深入理解每个计算步骤
- ✅ 完全控制前向和反向传播过程
- ✅ 学习底层数学原理
- ✅ 面试中展示对算法的理解

**PyTorch原生实现的优势：**
- ✅ 代码简洁，开发效率高
- ✅ 自动处理梯度计算和参数更新
- ✅ 内置优化和数值稳定性
- ✅ 易于扩展和维护
- ✅ 生产环境推荐使用

**学习建议：**
1. **先掌握自定义实现** - 理解底层原理
2. **再学习PyTorch原生** - 提高开发效率
3. **对比两种方式** - 加深理解
4. **实际项目用原生** - 保证代码质量


In [ ]:
# 性能对比分析 (Performance Comparison Analysis)

# 1. 训练时间对比
import time

print("⏱️  训练时间对比:")
print("-" * 50)

# 自定义实现训练时间（假设已经训练过）
custom_time = "已训练完成"
print(f"自定义实现: {custom_time}")

# PyTorch原生实现训练时间
pytorch_time = "已训练完成"  
print(f"PyTorch原生: {pytorch_time}")

print("\n📊 最终性能对比:")
print("-" * 50)

# 假设自定义实现的最终准确率（需要从之前的训练结果获取）
# 这里使用一个示例值，实际使用时需要从训练结果中获取
custom_final_acc = 0.95  # 请替换为实际的自定义模型准确率

print(f"自定义实现测试准确率: {custom_final_acc:.4f}")
print(f"PyTorch原生测试准确率: {final_acc:.4f}")

# 2. 代码复杂度对比
print("\n📝 代码复杂度对比:")
print("-" * 50)
print("自定义实现:")
print("- 需要手动实现前向传播")
print("- 需要手动计算梯度")
print("- 需要手动更新参数")
print("- 代码量: ~100行")

print("\nPyTorch原生实现:")
print("- 自动处理前向传播")
print("- 自动计算梯度")
print("- 自动更新参数")
print("- 代码量: ~30行")

print("\n🎯 学习收获:")
print("-" * 50)
print("✅ 理解了神经网络的底层原理")
print("✅ 掌握了梯度下降和反向传播")
print("✅ 学会了PyTorch的高效用法")
print("✅ 为面试和实际项目做好了准备")
